# Assignment for probabilistic models
## Seiyu CHOU 2022/2/2

## Question:
To verify the correctness of a Wikidata citation,
we need to search for fact-related sentences in the citation,
and sometimes the citation will be expressed using close synonyms of the wikidata predicate.
Using the search directly will return incorrect results.


## Solution:
Calculate the vector representation of quary and sentences in reference.
Compary and rank the similarity of sentences in the reference.

## Model and database:

I used BERT as the base model.
(https://huggingface.co/bert-base-uncased?text=Paris+is+the+%5BMASK%5D+of+France.)

STSbenchmark as the dataset to fine-tune it.
STSbenchmark worked for Semantic Textual Similarity task.
(http://ixa2.si.ehu.eus/stswiki/index.php/STSbenchmark)

The Training script is saved as training.py in project.


## import  model trained by STSbenchmark

In [24]:
#import model saved in output
from sentence_transformers import SentenceTransformer
import scipy

model  = SentenceTransformer('./output/training_stsbenchmark_continue_training-bert-base-uncased-2022-02-02_00-02-11')

## set example for sentences ranking and embedding
 the top 3 sentences came from reference of wikidata and the others are the Confusing Options

In [25]:
#set an example for sentence ranking
#embedding and print out
sentences = ['Son of Juan Rodríguez da Silva, who came from a Portuguese family that had settled in Andalusia in the early 16th century.',
        'Diego had a classical education, but preferred painting; he studied. under Francisco de Herrera the Elder.',
        'Velázquez was one of the most important European artists of the 17th century, spending his career from 1623 in the service of Philip IV of Spain.',
        'A man is eating a piece of bread.',
        'On the basis of a previous algorithm of cloud detection, clouds and aerosols are distinguished by analyzing the backscatter signals of a micro pulse lidar.',
        'A man is riding a horse.',
        'In this way, the content of clouds are classified into ice, water, mixed-phase clouds and horizontal oriented ice flakes. Finally, the supercooled water clouds are distinguished from water clouds by the information of temperature profiles.',
        'Two men pushed carts through the woods.',
        'A man is riding a white horse on an enclosed ground.',
        'A monkey is playing drums.',
        'A cheetah is running behind its prey.']
sentence_embeddings = model.encode(sentences)

for sentence, embedding in zip(sentences, sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)

Sentence: Lack of saneness
Embedding: [-2.90089518e-01  3.21964622e-02 -3.12187910e-01  4.72793043e-01
 -2.81635731e-01  6.45823181e-01  8.05791438e-01 -2.68952757e-01
 -6.20151579e-01  4.21721876e-01  2.20288202e-01 -1.30651861e-01
 -3.08907907e-02  3.83291006e-01  1.72608748e-01 -7.35940397e-01
 -6.63180202e-02 -7.89561793e-02  1.06581843e+00 -1.44582272e+00
  8.96878466e-02 -3.99974108e-01 -7.00156152e-01  9.09917802e-02
  5.18423438e-01 -9.94568646e-01 -3.08032691e-01 -1.91074774e-01
  8.22677370e-03  5.23903191e-01  9.02283415e-02 -4.38712537e-02
 -3.65755707e-01  1.55158490e-01 -4.42763180e-01  1.10840611e-01
  3.52673471e-01 -7.43870288e-02 -3.59270759e-02 -3.97357494e-01
 -7.57631838e-01  4.98753399e-01 -6.65556908e-01  9.91025984e-01
  2.16546193e-01 -2.83826679e-01  1.03583026e+00  2.11754963e-01
 -2.51464307e-01  6.49465501e-01  2.83317477e-01  1.67311102e-01
 -2.07377821e-01  2.84436136e-01 -1.26955599e-01 -2.65481889e-01
 -4.58460629e-01  6.62417293e-01 -2.37677559e-01  5.

## set quary and embedding
the quary came from wikidata's fact,this time we choose a painter from Spain:

Diego Velázquez (Q297)--given name(P735)--Diego(value)

the wikidata page :https://www.wikidata.org/wiki/Q297

the reference page :http://www.getty.edu/vow/ULANFullDisplay?find=&role=&nation=&subjectid=500016881

In [26]:
#set quary example
query = 'Diego Velázquez given name Diego'
queries = [query]
query_embeddings = model.encode(queries)
print("Query",query)
print("Embedding:", embedding)

Query Nobody has sane thoughts
Embedding: [-8.49447727e-01 -1.78215012e-01 -3.84382248e-01 -1.41303957e-01
  6.37484312e-01 -2.53352195e-01 -3.40287894e-01  5.44368327e-01
  4.05438691e-01 -2.93028593e-01 -2.29820848e-01 -5.43520927e-01
 -5.32496460e-02 -4.01003212e-01  7.52594888e-01 -1.44860041e+00
 -4.84464675e-01  1.11452556e+00 -4.88006510e-02 -1.36421692e+00
 -3.84237885e-01 -4.12155509e-01  2.05788970e-01 -2.15501264e-02
  3.73054415e-01  2.14317366e-01 -6.25824332e-01 -5.89738250e-01
 -1.71000123e-01 -1.40965831e+00 -1.11093245e-01  6.75009251e-01
 -3.85952264e-01 -9.73544344e-02  5.30036271e-01  3.14698577e-01
 -7.38874972e-01 -2.08613038e-01  9.64717045e-02  3.99794787e-01
 -1.47499338e-01 -3.10520470e-01  1.87785432e-01  1.03730965e+00
  1.13582872e-01  2.07258925e-01 -1.99667707e-01  8.73118579e-01
 -3.34767103e-01  7.50406742e-01 -4.88234550e-01 -4.00540978e-01
  7.55549490e-01 -4.63757157e-01  6.59310400e-01 -4.38424617e-01
 -6.74833536e-01  5.14566958e-01  3.79053444e-01

## calculate the cosine-similarits and print out

In [27]:
print("semantic Search Results")
number_top_matches = 5
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]
    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    print("Query:", query)
    print("\nTop {} most similar sentences in corpus:".format(number_top_matches))

    for idx, distance in results[0:number_top_matches]:
        print(sentences[idx].strip(), "(Cosine Score: %.4f)" % (1-distance))

semantic Search Results
Query: Nobody has sane thoughts

Top 3 most similar sentences in corpus:
Absence of sanity (Cosine Score: 0.5619)
Lack of saneness (Cosine Score: 0.1137)
A man is riding a white horse on an enclosed ground. (Cosine Score: 0.0817)


## Another result made by nli-distilroberta-base-v2


semantic Search Results
Query: Nobody has sane thoughts

Top 5 most similar sentences in corpus:

Son of Juan Rodríguez da Silva, who came from a Portuguese family that had settled in Andalusia in the early 16th century. (Cosine Score: 0.5346)

Velázquez was one of the most important European artists of the 17th century, spending his career from 1623 in the service of Philip IV of Spain. (Cosine Score: 0.4441)

Diego had a classical education, but preferred painting; he studied. under Francisco de Herrera the Elder. (Cosine Score: 0.3138)

A man is riding a horse. (Cosine Score: 0.1205)

A man is riding a white horse on an enclosed ground. (Cosine Score: 0.1019)

## Another result made by  msmarco-distilbert-base-v4


MS MARCO is a large scale information retrieval corpus that was created based on real user search queries using Bing search engine. The provided models can be used for semantic search, i.e., given keywords / a search phrase / a question, the model will find passages that are relevant for the search query.

The training data constist of over 500k examples, while the complete corpus consist of over 8.8 Million passages.

## semantic Search Results
Query: Nobody has sane thoughts

Top 5 most similar sentences in corpus:

Diego had a classical education, but preferred painting; he studied. under Francisco de Herrera the Elder. (Cosine Score: 0.4006)

Velázquez was one of the most important European artists of the 17th century, spending his career from 1623 in the service of Philip IV of Spain. (Cosine Score: 0.3903)

Son of Juan Rodríguez da Silva, who came from a Portuguese family that had settled in Andalusia in the early 16th century. (Cosine Score: 0.3529)

A man is riding a white horse on an enclosed ground. (Cosine Score: 0.1172)

A man is riding a horse. (Cosine Score: 0.0748)

## Sections that might can be optimized
1.train_batch_size could be bigger for better result.

2.the model msmarco-distilbert-base-v4 trained by bigger dataset and got more lower point for the correct answer.Maybe I could use another dataset to check the Influence